##### Copyright 2024 Google LLC.

Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at

    http://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.


# linear_programming_example

<table align="left">
<td>
<a href="https://colab.research.google.com/github/google/or-tools/blob/main/examples/notebook/linear_solver/linear_programming_example.ipynb"><img src="https://raw.githubusercontent.com/google/or-tools/main/tools/colab_32px.png"/>Run in Google Colab</a>
</td>
<td>
<a href="https://github.com/google/or-tools/blob/main/ortools/linear_solver/samples/linear_programming_example.py"><img src="https://raw.githubusercontent.com/google/or-tools/main/tools/github_32px.png"/>View source on GitHub</a>
</td>
</table>

First, you must install [ortools](https://pypi.org/project/ortools/) package in this colab.

In [ ]:
%pip install ortools


Linear optimization example.

In [7]:
from ortools.linear_solver import pywraplp

def solve_transportation_problem(costs, supply, demand):
    # Cân bằng cung và cầu
    total_supply = sum(supply)
    total_demand = sum(demand)

    if total_supply < total_demand:
        # Thêm một kho giả
        supply.append(total_demand - total_supply)
        costs.append([0] * len(demand))  # Chi phí từ kho giả đến các trạm = 0
    elif total_supply > total_demand:
        # Thêm một trạm giả
        demand.append(total_supply - total_demand)
        for cost in costs:
            cost.append(0)  # Chi phí từ các kho đến trạm giả = 0

    # Tạo solver
    solver = pywraplp.Solver.CreateSolver('GLOP')

    if not solver:
        print("Solver không khả dụng.")
        return

    num_warehouses = len(supply)  # Số lượng kho
    num_stations = len(demand)   # Số lượng trạm

    # Tạo biến quyết định x[i][j]: lượng xăng vận chuyển từ kho i đến trạm j
    x = []
    for i in range(num_warehouses):
        x.append([solver.NumVar(0, solver.infinity(), f'x[{i},{j}]') for j in range(num_stations)])

    # Ràng buộc cung ứng: tổng lượng xăng xuất khỏi mỗi kho không vượt quá cung cấp
    for i in range(num_warehouses):
        solver.Add(solver.Sum(x[i][j] for j in range(num_stations)) <= supply[i])

    # Ràng buộc nhu cầu: tổng lượng xăng nhận vào mỗi trạm đúng bằng nhu cầu
    for j in range(num_stations):
        solver.Add(solver.Sum(x[i][j] for i in range(num_warehouses)) == demand[j])

    # Hàm mục tiêu: tối thiểu hóa tổng chi phí vận chuyển
    solver.Minimize(solver.Sum(costs[i][j] * x[i][j] for i in range(num_warehouses) for j in range(num_stations)))

    # Giải bài toán
    status = solver.Solve()

    # Kiểm tra trạng thái giải pháp
    if status == pywraplp.Solver.OPTIMAL:
        print("Giải pháp tối ưu:")
        total_cost = 0
        for i in range(num_warehouses):
            for j in range(num_stations):
                if x[i][j].solution_value() > 0:
                    print(f"Chuyển {x[i][j].solution_value()} từ kho {i+1} đến trạm {j+1} với chi phí {costs[i][j]}")
                    total_cost += x[i][j].solution_value() * costs[i][j]
        print(f"Tổng chi phí: {total_cost}")
    else:
        print("Không tìm thấy giải pháp tối ưu.")

# Dữ liệu đầu vào
costs = [
    [30, 27, 26, 9, 23],  # Chi phí từ kho 1
    [13, 4, 22, 3, 1],   # Chi phí từ kho 2
    [3, 1, 5, 4, 24], # Chi phí từ kho 3
    [16, 30, 17, 10, 16],   # Chi phí từ kho 4
]
supply = [4, 6, 10, 10]   # Lượng xăng dự trữ tại các kho
demand = [7, 7, 7, 7, 2]  # Nhu cầu tại các trạm

solve_transportation_problem(costs, supply, demand)


Giải pháp tối ưu:
Chuyển 4.0 từ kho 1 đến trạm 4 với chi phí 9
Chuyển 4.0 từ kho 2 đến trạm 2 với chi phí 4
Chuyển 2.0 từ kho 2 đến trạm 5 với chi phí 1
Chuyển 7.0 từ kho 3 đến trạm 1 với chi phí 3
Chuyển 3.0 từ kho 3 đến trạm 2 với chi phí 1
Chuyển 7.0 từ kho 4 đến trạm 3 với chi phí 17
Chuyển 3.0 từ kho 4 đến trạm 4 với chi phí 10
Tổng chi phí: 227.0
